# Histograms and sequences of the visits in different tracts, patches and bands

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-05-30
- last update : 2025-06-01
- last update : 2025-06-04 : Better focus on 5063

In [ ]:
import lsst.pipe.base

print(lsst.pipe.base.__version__)

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
from lsst.geom import SpherePoint, degrees
from lsst.afw.image import ExposureF
from lsst.skymap import PatchInfo, Index2D
import numpy as np
import pandas as pd
from astropy.time import Time
#%matplotlib widget

In [ ]:
import seaborn as sns

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

### Choose instrument

In [ ]:
# instrument = "LSSTCam"
instrument = "LSSTComCam"
# instrument = "LATISS"

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- - Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

elif instrument == "LSSTComCamSim":
    repo = "/repo/main"
    collection_validation = "LSSTComCamSim/*"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "ops_rehersal_prep_2k_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

elif instrument == "LATISS":
    repo = "/repo/main"
    # collection_validation = instrument + "/runs/quickLook"
    collection_validation = instrument + "/raw/all"
    date_start = 20221001
    date_selection = 20221001
    skymapName = "latiss_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object

In [ ]:
# skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)

In [ ]:
try:
    skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)
except Exception as inst:
    print(type(inst))  # the exception type
    print(inst.args)  # arguments stored in .args
    print(inst)  # __str__ allows args to be printed directly,

## Dump registry into a pandas dataframe

- Faster method to decode the registry in pandas dataframe : first save deconded filed into a list of fields and then flush the whole list in pandas instead of row by row
- Be carefull the registry variable change in name and type perhaps depending on DM_version

In [ ]:
print(where_clause_date)

In [ ]:
columns = [
    "id",
    "obs_id",
    "day_obs",
    "seq_num",
    "time_start",
    "time_end",
    "type",
    "target",
    "filter",
    "zenith_angle",
    "expos",
    "ra",
    "dec",
    "skyangle",
    "azimuth",
    "zenith",
    "science_program",
    "jd",
    "mjd",
]

In [ ]:
df_exposure = pd.DataFrame(
    {
        "id": pd.Series(dtype="int"),
        "obs_id": pd.Series(dtype="int"),
        "day_obs": pd.Series(dtype="int"),
        "seq_num": pd.Series(dtype="int"),
        "time_start": pd.Series(dtype="str"),  # ou 'datetime64[ns]' si c’est un datetime
        "time_end": pd.Series(dtype="str"),  # idem
        "type": pd.Series(dtype="str"),
        "target": pd.Series(dtype="str"),
        "filter": pd.Series(dtype="str"),
        "zenith_angle": pd.Series(dtype="float"),
        "expos": pd.Series(dtype="float"),  # ou 'int' selon le cas
        "ra": pd.Series(dtype="float"),
        "dec": pd.Series(dtype="float"),
        "skyangle": pd.Series(dtype="float"),
        "azimuth": pd.Series(dtype="float"),
        "zenith": pd.Series(dtype="float"),
        "science_program": pd.Series(dtype="str"),
        "jd": pd.Series(dtype="float"),
        "mjd": pd.Series(dtype="float"),
    }
)

In [ ]:
# save the data array in rows before saving in pandas dataframe
rows = []
for count, info in enumerate(registry.queryDimensionRecords("exposure", where=where_clause_date)):
    try:
        jd_start = info.timespan.begin.value
        jd_end = info.timespan.end.value
        the_Time_start = Time(jd_start, format="jd", scale="utc")
        the_Time_end = Time(jd_end, format="jd", scale="utc")
        mjd_start = the_Time_start.mjd
        mjd_end = the_Time_end.mjd
        isot_start = the_Time_start.isot
        isot_end = the_Time_end.isot

        if count == 0:
            print("===== Time Conversion Debug Info =====")
            print(f"JD start      : {jd_start} (type: {type(jd_start)})")
            print(f"JD end        : {jd_end} (type: {type(jd_end)})")
            print(f"MJD start     : {mjd_start} (type: {type(mjd_start)})")
            print(f"MJD end       : {mjd_end} (type: {type(mjd_end)})")
            print(f"ISOT start    : {isot_start} (type: {type(isot_start)})")
            print(f"ISOT end      : {isot_end} (type: {type(isot_end)})")
            print("=======================================")

        # put row in a dictionnary before stacking
        row = {
            "id": info.id,
            "obs_id": info.obs_id,
            "day_obs": info.day_obs,
            "seq_num": info.seq_num,
            "time_start": isot_start,
            "time_end": isot_end,
            "type": info.observation_type,
            "target": info.target_name,
            "filter": info.physical_filter,
            "zenith_angle": info.zenith_angle,
            "expos": info.exposure_time,  # Exemple : adapter selon ton objet
            "ra": info.tracking_ra,
            "dec": info.tracking_dec,
            "skyangle": info.sky_angle,
            "azimuth": info.azimuth,
            "zenith": info.zenith_angle,
            "science_program": info.science_program,
            "jd": float(jd_start),
            "mjd": float(mjd_start),
        }
        rows.append(row)

    except ValueError as e:
        print(f"Erreur de valeur : {e}")
    except FileNotFoundError as e:
        print(f"Fichier introuvable : {e}")
    except Exception as e:
        print(f"Erreur inattendue : {type(e).__name__} - {e}")
        print(f">>>   Unexpected error at row {count}:", sys.exc_info()[0])
        traceback.print_exc()  # affiche la stack trace complète

In [ ]:
# Création finale du DataFrame
df_exposure = pd.DataFrame(rows)

In [ ]:
df_exposure

In [ ]:
# df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

## Select science exposures

In [ ]:
df_science = df_exposure[df_exposure.type == "science"]
df_science.reset_index(drop=True, inplace=True)

## Add Tract-Patches

In [ ]:
df=df_science.copy()
df["band"] = df["filter"].apply(lambda x : x.split("_")[0])

In [ ]:
#df_with_tract_patch = add_tract_patch(df, butler)

In [ ]:
def get_tract_patch(row, skymap):
    if pd.isna(row['ra']) or pd.isna(row['dec']):
        return pd.Series({"tract": None, "patch": None})
    
    target_point = SpherePoint(row['ra'], row['dec'], degrees)

    tract_info = skymap.findTract(target_point)
    patch_info = tract_info.findPatch(target_point)
    tractNbSel = tract_info.getId()
    patchNbSel =  patch_info.getSequentialIndex()
    patch_index_str = f"{patch_info.getIndex()[0]},{patch_info.getIndex()[1]}"
   
    
    return pd.Series({"tract":   tractNbSel, "patch":  patchNbSel, "patch_str": patch_index_str})


In [ ]:
df = df.copy()
df[['tract', 'patch', "patch_str"]] = df.apply(get_tract_patch, axis=1, args=(skymap,))

In [ ]:
df

In [ ]:
df["tag"] = df["tract"].astype(str) + "_" + df["target"]

In [ ]:
df['tag'].unique()

In [ ]:
df["target"].unique()

In [ ]:
df["science_program"].unique()

## Start plotting

### Plot tags

In [ ]:
# 1. Trier le DataFrame par numéro de tract
df = df.sort_values("tract")

In [ ]:
# 2. Créer la colonne tag après le tri
df["tag"] = df["tract"].astype(str) + "_" + df["target"]

In [ ]:
# 3. Grouper par tag et band, et compter
grouped_tag = df.groupby(['tag', 'band']).size().reset_index(name='count')

# 4. Définir l'ordre des tags selon l'ordre dans df trié
tag_order = df["tag"].drop_duplicates().tolist()

In [ ]:
# Force band order
band_order = ['u', 'g', 'r', 'i', 'z', 'y']
color_map = {
    'u': 'blue',
    'g': 'green',
    'r': 'red',
    'i': 'orange',
    'z': 'gray',
    'y': 'black'
}

In [ ]:
# 6. Tracer le barplot
plt.figure(figsize=(20, 8))
sns.barplot(
    data=grouped_tag,
    x='tag',
    y='count',
    hue='band',
    hue_order=band_order,
    palette=color_map,
    order=tag_order  # ordre des tags triés par tract
)

plt.xlabel("Visited fields / tract")
plt.ylabel("Number of visits per band")
plt.title("Number of visits per band in each tract",fontsize=20,fontweight="bold")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Band")
plt.tight_layout()
plt.show()


In [ ]:
# 6. Tracer le barplot
plt.figure(figsize=(8, 20))
sns.barplot(
    data=grouped_tag,
    x='count',
    y='tag',
    hue='band',
    hue_order=band_order,
    palette=color_map,
    order=tag_order  # ordre des tags triés par tract
)

plt.ylabel("Visited fields / tract")
plt.xlabel("Number of visits per band")
plt.title("Number of visits per band in each tract",fontsize=20,fontweight="bold")
#plt.xticks(rotation=45, ha='right')
plt.legend(title="Band")
plt.tight_layout()
plt.show()

### Focus in selected TRACTSEL

In [ ]:
df_tract = df[df["tract"] == TRACTSEL]

# 3. Grouper par tag et band, et compter
grouped_tag_tract = df_tract.groupby(['tag', 'band']).size().reset_index(name='count')

# 4. Définir l'ordre des tags selon l'ordre dans df trié
tag_order = df_tract["tag"].drop_duplicates().tolist()

In [ ]:
# 6. Tracer le barplot
plt.figure(figsize=(10, 6))
sns.barplot(
    data=grouped_tag_tract,
    x='tag',
    y='count',
    hue='band',
    hue_order=band_order,
    palette=color_map,
    order=tag_order  # ordre des tags triés par tract
)

plt.xlabel("Visited fields / tract")
plt.ylabel("Number of visits per band")
plt.title("Number of visits per band in each tract",fontsize=20,fontweight="bold")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Band")
plt.tight_layout()
plt.show()


### Plots histograms with the number of visits in tract and patches

In [ ]:
# Grouper et compter
grouped = df.groupby(['tract', 'patch', 'band']).size().reset_index(name='count')

In [ ]:
#grouped['tract_patch'] = grouped['tract'].astype(str) + '_' + grouped['patch_str']
grouped['tract_patch'] = grouped['tract'].astype(str) + '_' + grouped['patch'].astype(str)

In [ ]:
plt.figure(figsize=(20, 6))
sns.barplot(
    data=grouped,
    x='tract_patch',
    y='count',
    hue='band',
    hue_order=band_order,
    palette=color_map
)

plt.xlabel("Tract_Patch")
plt.ylabel("Number of visits per bands")
plt.title("Number of visits per bands in each (tract, patch)",fontsize=20,fontweight="bold")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Band")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, max(6, len(grouped['tract_patch'].unique()) * 0.8)))  # Adapter la hauteur

sns.barplot(
    data=grouped,
    y='tract_patch',  # axe Y = les (tract, patch)
    x='count',        # axe X = nombre de visites
    hue='band',
    hue_order=band_order,
    palette=color_map
)

plt.ylabel("Tract_Patch")
plt.xlabel("Number of visits per bands")
plt.title("Number of visits per bands in field of view (tract, patch)")
plt.legend(title="Band", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()
plt.show()


### Select tractsel

In [ ]:
df_tract = df[df["tract"] == TRACTSEL]
df_tract = df_tract.sort_values("patch")

# Grouper et compter
grouped_tag_tract= df_tract.groupby(['tract', 'patch', 'band']).size().reset_index(name='count')

grouped_tag_tract['tract_patch'] = grouped_tag_tract['tract'].astype(str) + '_' + grouped_tag_tract['patch'].astype(str)


# 4. Définir l'ordre des tags selon l'ordre dans df trié
tag_order = df_tract["tag"].drop_duplicates().tolist()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(
    data=grouped_tag_tract,
    x='tract_patch',
    y='count',
    hue='band',
    hue_order=band_order,
    palette=color_map
)

plt.xlabel("Tract_Patch")
plt.ylabel("Number of visits per bands")
plt.title(f"Number of visits per bands in each ({TRACTSEL}, patch)",fontsize=20,fontweight="bold")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Band")
plt.tight_layout()
plt.show()

## Which tract,patch has the maximum number of y visits

In [ ]:
grouped

In [ ]:
# Filtrer la bande 'y'
df_y = df[df['band'] == 'z']

# Grouper par tract et patch
grouped_y = df_y.groupby(['tract', 'patch']).size().reset_index(name='count')

# Trouver le max
max_row = grouped_y.loc[grouped_y['count'].idxmax()]

tract_max = max_row['tract']
patch_max = max_row['patch']

In [ ]:
print(f"Tract: {max_row['tract']}, Patch: {max_row['patch']}, Nombre de visites en bande y: {max_row['count']}")

## Time sequences of arrival time in selected tract-patch

In [ ]:
df_sel = df[(df['tract'] == tract_max) & (df['patch'] == patch_max)]

In [ ]:
plt.figure(figsize=(16, 6))

sns.stripplot(
    data=df_sel,
    x='mjd',
    y='band',
    hue='band',  # <- correction
    order=band_order,
    palette=color_map,
    size=10,
    alpha=0.7,
    dodge=False,
    legend=False  # <- évite de doubler les légendes
)

plt.xlabel("MJD")
plt.ylabel("Band")
plt.title(f"Sequence of visits in tract={tract_max}, patch={patch_max}",fontsize=20,fontweight="bold")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(16, 6))

# Stripplot : points temporels
sns.stripplot(
    data=df_sel,
    x='mjd',
    y='band',
    hue='band',
    order=band_order,
    palette=color_map,
    size=12,
    alpha=0.6,
    dodge=False,
    legend=False
)

# KDE : densité temporelle pour chaque bande
for band in band_order:
    band_data = df_sel[df_sel['band'] == band]
    if len(band_data) > 1:  # KDE a besoin de plus d'un point
        sns.kdeplot(
            data=band_data,
            x='mjd',
            bw_adjust=0.5,
            color=color_map[band],
            label=f"Density {band}",
            fill=False,
            linewidth=1.5
        )

plt.xlabel("MJD")
plt.ylabel("Band",fontsize=20,fontweight="bold")
plt.title(f"Sequence of visits + density per band in tract={tract_max}, patch={patch_max}",fontsize=20,fontweight="bold")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:

bands_present = [b for b in band_order if b in df_sel['band'].unique()]
n_bands = len(bands_present)

fig, axes = plt.subplots(n_bands, 1, figsize=(18, 1.5 * n_bands), sharex=True)

for i, band in enumerate(bands_present):
    ax = axes[i]
    band_data = df_sel[df_sel['band'] == band]

    # KDE plot (densité)
    if len(band_data) > 1:
        sns.kdeplot(
            data=band_data,
            x='mjd',
            bw_adjust=0.5,
            fill=True,
            color=color_map[band],
            alpha=0.3,
            ax=ax
        )

    # Rugplot = traits verticaux aux positions des observations
    sns.rugplot(
        data=band_data,
        x='mjd',
        color=color_map[band],
        height=0.5,
        linewidth=3,
        ax=ax
    )

    ax.set_ylabel(band, rotation=0, labelpad=20, fontsize=18, weight='bold')
    ax.grid(True, linestyle='--', alpha=0.3)
    ax.set_yticks([])

axes[-1].set_xlabel("MJD")
fig.suptitle(f"Séquence temporelle et densité par bande\n(tract={tract_max}, patch={patch_max})", fontsize=20,fontweight="bold")
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

bands_present = [b for b in band_order if b in df_sel['band'].unique()]
n_bands = len(bands_present)

fig, axes = plt.subplots(n_bands, 1, figsize=(18, 1.5 * n_bands), sharex=True)

for i, band in enumerate(bands_present):
    ax = axes[i]
    band_data = df_sel[df_sel['band'] == band]
    mjds = band_data['mjd'].dropna()

    # KDE plot (densité)
    if len(mjds) > 1:
        sns.kdeplot(
            x=mjds,
            bw_adjust=0.5,
            fill=True,
            color=color_map[band],
            alpha=0.3,
            ax=ax
        )

    # Rugplot
    sns.rugplot(
        x=mjds,
        color=color_map[band],
        height=0.2,
        linewidth=2,
        ax=ax
    )

    # Histogramme discret
    ax.hist(
        mjds,
        #bins='auto',                    # nombre de barres (modifiable)
        bins=30,                    # nombre de barres (modifiable)
        color=color_map[band],
        alpha=0.2,
        edgecolor=color_map[band],
        linewidth=1.0,
    )

    ax.set_ylabel(band, rotation=0, labelpad=20, fontsize=18, weight='bold')
    ax.grid(True, linestyle='--', alpha=0.3)
    ax.set_yticks([])

axes[-1].set_xlabel("MJD")
fig.suptitle(f"Séquence temporelle, densité & histogramme par bande\n(tract={tract_max}, patch={patch_max})", fontsize=20,fontweight="bold")
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()
